In [ ]:
import os
import sys
import chess
import chess.svg
from IPython.display import SVG, display, clear_output

# Add the src directory to the Python path
src_path = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', 'src'))
sys.path.append(src_path)

from chess_zero.agent.player_chess import ChessPlayer
from chess_zero.config import Config, PlayWithHumanConfig
from chess_zero.env.chess_env import ChessEnv
from chess_zero.agent.model_chess import ChessModel
from chess_zero.lib.model_helper import load_best_model_weight

def get_player(config):
    model = ChessModel(config)
    if not load_best_model_weight(model):
        raise RuntimeError("Best model not found!")
    return ChessPlayer(config, model.get_pipes(config.play.search_threads))

def display_board(board):
    clear_output(wait=True)
    display(SVG(chess.svg.board(board=board, size=400)))

def play_game():
    default_config = Config()
    PlayWithHumanConfig().update_play_config(default_config.play)
    ai_player = get_player(default_config)
    env = ChessEnv().reset()

    while not env.done:
        display_board(env.board)
        
        if env.board.turn == chess.WHITE:  # Human plays as White
            while True:
                try:
                    move = input("Enter your move (e.g., e2e4): ")
                    env.step(move)  # Pass the string directly
                    break
                except ValueError:
                    print("Invalid move. Try again.")
        else:  # AI plays as Black
            action = ai_player.action(env, False)
            print(f"AI move: {action}")
            env.step(action)
    
    display_board(env.board)
    print("Game Over")
    print(f"Result: {env.board.result()}")

# Run the game`
play_game()